# Compute und Hops - Ein kurzer Überblick

## Themen:
- “Das kopflose Rhino”: Der Compute-Server
- “Hopfen”: Verwendung und Möglichkeiten
- “Hops als Schnittstelle zur Außenwelt”: Der Python-Server
- “Von außen nach Innen”: Compute und Grasshopper

Leider sind die Beispiele und Tutorials aktuell nicht wirklich gepflegt und viele Links funktionieren nicht...

***

## Voraussetzungen
- Python > 3.9
- IDE, z.B. Visual Studio Code
- Rhino > 3.19.20
- Idealerweise eine virtuelle Entwicklungsumgebung (z.B. Anaconda)

Alles weitere gehen wir gemeinsam durch.

***

Für einige Dinge wird nicht zwangsweise der Compute- Server benötigt.
Es steht ebenfalls die __[OpenNURBS](https://www.rhino3d.com/de/opennurbs/)- Biliothek zur Verfügung. 
Wir verwenden Sie für die Definition von Variablen, zum Lesen und Schreiben von .3dm Dateien und für einfache Geometrie- Operationen

In [1]:
import rhino3dm

***



## Rhino Compute

### Was ist Compute?
- Im Grunde ein Rhino ohne GUI 
- Eine auf einem Webserver laufende Instanz von Rhino, über die man mittels einer REST-API kommunizieren kann.
- Rhino Compute bietet den gleichen Funktionsumfang, wie die "nomale" Rhino Version. --> Es können dementsprechend auch Grasshopper- Skripte ausgeführt werden.

#### Vorteile:
- Open Source!
- Ermöglicht parallele Ausführung von Befehlen.
- Berechnung kann auf einem externen Server ausgeführt werden.
- Kommunikation mit dem Server mittels Python, JavaScript oder .NET --> Damit bilden Rhino und Grasshopper nur noch ein Teilprogramm des Gesamtsystems.
- Rhino kann sich damit nun auf seine Stärken konzentrieren, die Verarbeitung von Geometrien.

#### Installation:
__[Installations Anleitung](https://developer.rhino3d.com/guides/compute/development/)__

__[GitHub](https://github.com/mcneel/compute.rhino3d)__

1. Klonen des Repos 
2.  src\compute.sln in Visual Studio öffnen und compilieren als DEBUG.
3. Sicherstellen, dass compute.geometry das Start- Projekt ist.
4. Ausführen der compute.geometry.exe
5. Über __[http://localhost:8081/version](http://localhost:8081/version)__ prüfen, ob der Server läuft.



In [2]:
import compute_rhino3d

***

Tipp: Man achte auf den Unterschied zwischen 
<br>
`import compute_rhino3d`    
und    
`pip install compute-rhino3d`

***

##### Ein kleines Anwendungsbeispiel:
Imports:

In [1]:
from rhino3dm import *
import tkinter as Tkinter
import compute_rhino3d.Mesh
import compute_rhino3d.Util
import requests
import json
import time

Verbindung zum Server definieren:

In [2]:
compute_rhino3d.Util.url = "http://localhost:5000/"
compute_rhino3d.Util.apiKey = ""

Verbindung prüfen:

In [4]:
try: 
    version_test = requests.get(compute_rhino3d.Util.url + '/version')
    version = json.loads(version_test.content)
    print("[INFO] Successfull: Server-version: ", version)
except:
    print("[ERROR] Can't reach the server!")


[INFO] Successfull: Server-version:  {'rhino': '7.20.22178.1001', 'compute': '1.0.0.0', 'git_sha': None}


Verwendung der OpenNURBS- Bibliothek:

In [5]:
center = Point3d(250, 250, 0)
sphere = Sphere(center, 100)
brep = sphere.ToBrep()

Folgender Befehl steht nicht in der Bibliothek zur Verfügung, also verwenden wir Rhino-Compute:

In [6]:
d = compute_rhino3d.Mesh.CreateFromBrep(brep)
mesh = (d[0])

Anzeigen des Ergebnisses:

In [7]:
top = Tkinter.Tk()

w = Tkinter.Canvas(top, width=500, height=500)
w.pack()

v = Vector3d(1,1,0)
for x in range(100):
    top.update()
    time.sleep(0.001)
    w.delete("all")
    mesh.Rotate(.1, v, center)
    verts = mesh.Vertices
    faces = mesh.Faces
    for i in range(len(faces)):
        face = faces[i]
        pts = [verts[face[0]], verts[face[1]], verts[face[2]], verts[face[3]]]
        w.create_line(pts[0].X, pts[0].Y, pts[1].X, pts[1].Y)
        w.create_line(pts[1].X, pts[1].Y, pts[2].X, pts[2].Y)
        w.create_line(pts[2].X, pts[2].Y, pts[3].X, pts[3].Y)
        w.create_line(pts[3].X, pts[3].Y, pts[0].X, pts[0].Y)

top.mainloop()

TclError: invalid command name ".!canvas"

Soweit so gut. Verwenden tut man es so eher selten (meiner Meinung nach), da die Entwicklung sehr aufwendig ist.
<br>
Deshalb machen wir einen kleinen Sprung:

***

## Hops
### Was ist Hops und was kann es?
- Plugin für Grasshopper.
- Auf den ersten Blick ähnlich zu den Clustern.
- Allerdings ist es mehr eine Schnittstelle.
- Besteht aus dem Hops-Component selbst und einigen In- und Outputs.
- Teilen von Definitionen über verschiedene Projekte hinweg.
- Parallele und Asynchrone Berechnung. Der Solver wird nicht mehr geblockt.
- Rekursionen sind möglich!

### Installation
Über den Package-Manager

Weiter geht es in Grasshopper

***

## Das Hops Plugin als Schnittstelle - Jetzt kommt der spannende Teil
Dies kann auf zwei unterschiedliche Arten erfolgen:
1) Ein eigenes Plugin definieren und Parameter übergeben/ zurückgeben --> Teilautomatisierung
2) Grasshopper Definitionen direkt auf dem Compute- Server ausführen --> Vollständige Automatisierung

Samples: __[GitHub](https://github.com/mcneel/rhino-developer-samples/tree/7/compute)__

### Der Python- Server

Imports

In [8]:
from flask import Flask
import ghhops_server as hs
import rhino3dm

Hops als Middleware registrieren

In [9]:
app = Flask(__name__)
hops: hs.HopsFlask = hs.Hops(app)

Definition des Components und direkt im Anschluss die auszuführende Funktion.

In [17]:
@hops.component(
    "/add",
    name="Add",
    nickname="Add",
    description="Add numbers with CPython",
    icon = "C:\\Users\\Johannes\\Documents\\GitHub\\hops_tutorial\\recources\\icon_add.png",
    inputs=[
        hs.HopsNumber("A", "A", "First number"),
        hs.HopsNumber("B", "B", "Second number")
    ],
    outputs=[
        hs.HopsNumber("Sum", "S", "A + B")
    ]
)
def add(a: float, b: float):
    c = a + b
    return c

Es können auch mehrere Funktionen innerhalb einer Definition erstellt werden:
Und ein Tree-Access muss extra definiert werden!

In [ ]:
@hops.component(
    "/pointat",
    name="PointAt",
    nickname="PtAt",
    description="Get point along curve",
    inputs=[
        hs.HopsCurve("Curve", "C", "Curve to evaluate"),
        hs.HopsNumber("t", "t", "Parameter on Curve to evaluate", hs.HopsParamAccess.TREE)
    ],
    outputs=[hs.HopsPoint("P", "P", "Point on curve at t")]
)
def pointat(curve: rhino3dm.Curve, t=0.0):
    return curve.PointAt(t)

Starten des Flask- Servers

In [ ]:
if __name__ == "__main__":
    app.run(debug=True)

***

## Grasshopper Definitionen mittels Compute lösen
- Verarbeitung vollständig serverseitig.
- Volle automatisierung
- Kein Grasshopper mehr notwendig zum ausführen.

In [10]:
# Imports:
import requests
import json
import os
import sys
import compute_rhino3d.Util as rcutil
import compute_rhino3d.Grasshopper as rcgh

Wie gehabt, Verbindung aufbauen und testen:

In [11]:
# Set the compute server connection
compute_url = "http://localhost:5000/"
post_url = compute_url + "grasshopper"

dir_path = os.path.dirname(os.path.realpath(__file__))
ghDefinition_path = os.path.join(dir_path, 'ghCompute.gh')

rcutil.url = compute_url
rcutil.authToken = "" # no auth token required as it is a local service

# Test the compute server connection, should return version object
print("T")
try: 
    version_test = requests.get(compute_url + '/version')
    print("[INFO] Compute server connection established")
except:
    print("[ERROR] Can not establish a connection to the compute server!")
    sys.exit(1)

NameError: name '__file__' is not defined

Nun müssen die Inputs definiert werden:

In [ ]:
inputTree1 = rcgh.DataTree("RH_IN:number1")
inputTree2 = rcgh.DataTree("number2")

Variablen mit Infos füllen:
Folgendes ist hierbei zu beachten:

Der Branch wird als Liste übergeben. Aus [0,1,0] wird später {0;1;0}.
Anschließend wird für jeden Branch eine Liste an Items übergeben.

In [ ]:
#Filling the trees:
for i in range(1,6):   #1,2,3,4,5 
    inputTree1.Append([i], [i]) #Beides müssen listen sein!


inputTree2.Append([0], [3])

# Merge all inputs:
trees = [inputTree1, inputTree2]

Grasshopper Definition ausführen:

In [ ]:
output = rcgh.EvaluateDefinition(ghDefinition_path, trees)

Das Ergebnis kommt im JSON- Format zurück und kann nach bekannten Methoden verarbeitet werden

In [ ]:
print(json.dumps(output,indent=4))